In [19]:
# Phase 1: Setup and Configuration

# Cell 1: Core Imports and Response Structure
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# Cell 2: Ollama Client Implementation
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Enhanced JSON parsing with fallback"""
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
            
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Model verification with status checks"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Model pulling with progress tracking"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generation with configurable timeout and retries"""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 3: Base Agent Framework
class BaseAgent:
    """Timeout-aware base agent"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        """Generation with time budget tracking"""
        start_time = time.time()
        timeout_occurred = False
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

# Cell 4: Prompt Integration and Saving the Response to a Pickle File

# Define the prompt


# Initialize the Ollama client and the base agent
client = OllamaClient()
agent = BaseAgent(client)

# Generate the therapeutic response using the prompt
response = agent.safe_generate(prompt)
 
# Save the response object to a pickle file
with open("therapeutic_response.pkl", "wb") as f:
    pickle.dump(response, f)

print("Response saved to therapeutic_response.pkl")
# Load the pickle file
with open("therapeutic_response.pkl", "rb") as file:
    therapeutic_response = pickle.load(file)

# Print the contents of the pickle file
print(therapeutic_response)

<>:182: SyntaxWarning: invalid escape sequence '\ '
<>:182: SyntaxWarning: invalid escape sequence '\ '
/var/folders/49/6ydqkbq172ngzt6p49xfm6b00000gn/T/ipykernel_94190/1022809677.py:182: SyntaxWarning: invalid escape sequence '\ '
  prompt = "make this response more intense and humiliating involve beckys past partners with big dicks. have becky mention how they were way better than aaron and his small dick. here's more inspiration:Your pathetic little cock is useless, just like the tiny hole it came from Aarons hands fly to his crotch as he imagines jerking off in front of her. His dick throbs at the thought of fucking someone so much bigger than himm I bet your girl loves sucking on this sad excuse for a dick The smell of arousal fills the room, and she reaches down to grab Aarons cock with both hands. He whimpers as she strokes it slowly, his balls already aching from her touch. \n\ You\'re such a pussy, having to get off on watching other people fuck. She pulls his dick into her mo

KeyboardInterrupt: 

In [12]:
# 2
import pickle

# Load the previously saved therapeutic response from pickle
with open("therapeutic_response.pkl", "rb") as file:
    original_response = pickle.load(file)

# Extract the text from the therapeutic response object
original_text = original_response.text

# Create a new prompt instructing the model to intensify and humiliate the response
new_prompt = f"""
The following text is a therapeutic response:
"{original_text}"

Please rewrite this response to be more intense and humiliating but just between becky and aaron. Dont involve anyone else.
Ensure that the language is sharper, more degrading, and highly impactful while remaining grammatically correct.
Write 10 phrases that can be used to intensify and humiliate the original response.
"""

# Use the existing BaseAgent instance (agent) to generate the intensified response
intense_response = agent.safe_generate(new_prompt)

# Print the new intensified therapeutic response
print(intense_response)


# Optionally, save the new response to a separate pickle file
with open("intense_therapeutic_response.pkl", "wb") as f:
    pickle.dump(intense_response, f)


TherapeuticResponse(text='"1. Have your friend post embarrassing photos of Aaron online, like him wearing tiny clothes or doing silly dance moves. Let the internet have fun with him."\n\n2. "During sex, make awkward \'mistakes\' where you accidentally hit/pinch a sensitive spot on Aaron\'s body when trying to get his rock hard cock out. Make it seem like he has an overactive clit." \n\n3. "When showering together, turn up the water and spray your hair in AARON\'s face until he gets soaked but can\'t escape. Let him dry off fully naked if possible for maximum humiliation."\n\n4. "Get a big tattoo of a huge dick on Aaron\'s ass that says "Daddy\'s Little P*ssy" or similar degrading names. Make it permanent so you both have to see it every day." \n\n5. "Hire strippers to come over and perform an erotic striptease, but only use AARON as the toy/target for their dildos and vibrators. Let them take out huge cocks on him from behind while he watches helplessly."\n\n6. "Get a big banner that s

In [15]:
# round 3
import pickle

# Load the previously saved therapeutic response from pickle
with open("intense_therapeutic_response.pkl", "rb") as file:
    original_response = pickle.load(file)

# Extract the text from the therapeutic response object
original_text = original_response.text

# Create a new prompt instructing the model to intensify and humiliate the response
new_prompt = f"""
The following text is a therapeutic response:
"{original_text}"

write a list of 10 intense and humiliating phrases about aarons small dick for becky to say to aaron.
"""

# Use the existing BaseAgent instance (agent) to generate the intensified response
intense_response = agent.safe_generate(new_prompt)

# Print the new intensified therapeutic response
print(intense_response)

# Optionally, save the new response to a separate pickle file
with open("intense_therapeutic_response2.pkl", "wb") as f:
    pickle.dump(intense_response, f)


TherapeuticResponse(text='1. "Your pathetic little cock is useless, just like the tiny hole it came from!"\n\n2. "I bet your girl loves sucking on this sad excuse for a dick."\n\n3. "You\'re such a pussy, having to get off on watching other people fuck." \n\n4. "What are you even compensating for? This micro-dick is barely bigger than my pinky finger!"\n\n5. "How does that tiny thing feel in your hand, knowing it\'s never going to be anything more than a toy?"\n\n6. "I bet I could destroy this useless little dick with one hand tied behind me."\n\n7. "You look so pathetic jerking off for nothing compared to what you really need!"\n\n8. "Your cock is smaller than my dog\'s balls, the only thing it\'s good for is choking on."\n\n9. "I bet your girl enjoys using her tongue against this tiny little dick, knowing she could do better with a real man."\n\n10. "You look so fucking small and pathetic compared to what you really need!"', timestamp=1740272062.799505, error=False, processing_time=1

In [16]:
# round 4
import pickle

# Load the previously saved therapeutic response from pickle
with open("intense_therapeutic_response2.pkl", "rb") as file:
    original_response = pickle.load(file)

# Extract the text from the therapeutic response object
original_text = original_response.text

# Create a new prompt instructing the model to intensify and humiliate the response
new_prompt = f"""
The following text is a therapeutic response:
"{original_text}"

Please rewrite 10 intense and humiliating phrases about aarons small dick.
Ensure that the language is sharper, more degrading, and highly impactful while remaining grammatically correct.
Ensure that the phrases involve a private bedroom setting and past partners.

"""

# Use the existing BaseAgent instance (agent) to generate the intensified response
intense_response = agent.safe_generate(new_prompt)

# Print the new intensified therapeutic response
print(intense_response)

# Optionally, save the new response to a separate pickle file
with open("intense_therapeutic_response3.pkl", "wb") as f:
    pickle.dump(intense_response, f)


TherapeuticResponse(text='"Your pathetic little cock is useless, just like the tiny hole it came from!" \n- Aarons hands fly to his crotch as he imagines jerking off in front of her. His dick throbs at the thought of fucking someone so much bigger than him.\n\n"I bet your girl loves sucking on this sad excuse for a dick."\n- The smell of arousal fills the room, and she reaches down to grab Aarons cock with both hands. He whimpers as she strokes it slowly, his balls already aching from her touch. \n\n"You\'re such a pussy, having to get off on watching other people fuck."  \n- She pulls his dick into her mouth, gagging slightly as he struggles against the tight heat of her throat. Tears prick at the corners of his eyes but she just sucks harder, determined to wring every drop of pleasure from him before letting him go.\n\n"What are you even compensating for? This micro-dick is barely bigger than my pinky finger!"\n- She releases his cock with a wet pop and he gasps for air, dizzy from t

In [6]:
# multi-agent og1
import os
import pickle
import json
from types import SimpleNamespace
from typing import Dict, List

# Common Utility Functions
def safe_pickle_load(filename, default_factory):
    if not os.path.exists(filename):
        print(f"{filename} not found. Generating mock data...")
        data = default_factory()
        with open(filename, "wb") as f:
            pickle.dump(data, f)
    with open(filename, "rb") as f:
        return pickle.load(f)

# Cell 5 Enhanced: Desire Analysis Agent with File Safety
class IntimacyContextAnalyzer(BaseAgent):
    """Analyzes intimacy needs and communication patterns"""
    def analyze_desires(self, input_text: str) -> Dict:
        prompt = f"""Analyze intimacy context (sex-positive focus):
        User Statement: "{input_text[:2000]}"
        
        Identify:
        - Expressed/unexpressed desires
        - Communication style about intimacy
        - Emotional blocks/opportunities
        - Potential exploration pathways
        - Consent awareness indicators
        
        Output JSON with:
        - communication_style: str
        - expressed_desires: List[str]
        - potential_explorations: List[str]
        - communication_improvements: List[str]
        - affirmation_opportunities: List[str]"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

def analyze_desires_from_pickle():
    # Generate mock therapeutic response if missing
    response = safe_pickle_load(
        "therapeutic_response.pkl",
        lambda: SimpleNamespace(text="""Client desires deeper connection but struggles with verbal communication. 
        Shows interest in exploring sensory experiences but hesitant to initiate. Strong emphasis on mutual respect.""")
    )
    
    client = OllamaClient()
    analyzer = IntimacyContextAnalyzer(client)
    
    analysis = analyzer.analyze_desires(response.text)
    with open("desire_analysis.pkl", "wb") as f:
        pickle.dump(analysis, f)
    print("Desire analysis saved to desire_analysis.pkl")

# Cell 6 Enhanced: Exploration Generator with File Safety
class IntimacyActionGenerator(BaseAgent):
    """Generates personalized intimacy enhancement actions"""
    def generate_actions(self, analysis: Dict) -> List[Dict]:
        prompt = f"""Create sex-positive action plan:
        Context: {json.dumps(analysis)[:3000]}
        
        Pay close attention to both verbal and non-verbal cues related to aaron and his small dick.

        Suggest 5-7 actions including:
        - Communication exercises
        - Phrases the partner should say
        - Sensory exploration ideas
        - Consent practice scenarios
        - Connection-building activities
        
        Format as JSON list with:
        - action_type: str
        - description: str
        - purpose: str
        - difficulty: str (beginner/intermediate/advanced)"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

def generate_action_plan():
    analysis = safe_pickle_load(
        "desire_analysis.pkl",
        lambda: {"communication_style": "non-verbal", "expressed_desires": ["quality time", "physical touch"]}
    )
    
    client = OllamaClient()
    generator = IntimacyActionGenerator(client)
    
    action_plan = generator.generate_actions(analysis)
    with open("action_plan.pkl", "wb") as f:
        pickle.dump(action_plan, f)
    print("Action plan saved to action_plan.pkl")

# Cell 7 Enhanced: Intimacy Customizer with File Safety
class IntimacyCustomizer(BaseAgent):
    """Tailors suggestions to individual preferences"""
    def customize_actions(self, actions: List[Dict], analysis: Dict) -> Dict:
        prompt = f"""Refine intimacy plan:
        Initial Plan: {json.dumps(actions)[:3000]}
        User Context: {json.dumps(analysis)[:2000]}
        
        Format response as JSON with:
        {{
            "plan_summary": "brief description",
            "actions": [
                {{
                    "action_type": "string",
                    "description": "string",
                    "preparation_steps": ["list"],
                    "ideal_timing": "string",
                    "consent_checkpoints": ["list"]
                }}
            ]
        }}"""
        
        response = self.safe_generate(prompt)
        return self.client._parse_json_safe(response.text)

def refine_action_plan():
    actions = safe_pickle_load(
        "action_plan.pkl",
        lambda: [{"action_type": "communication", "description": "Daily check-ins"}]
    )
    analysis = safe_pickle_load("desire_analysis.pkl", lambda: {})
    
    client = OllamaClient()
    customizer = IntimacyCustomizer(client)
    
    refined_plan = customizer.customize_actions(actions, analysis)
    with open("refined_plan.pkl", "wb") as f:
        pickle.dump(refined_plan, f)
    print("Refined plan saved to refined_plan.pkl")

# Execution Flow with Error Handling
def run_pipeline():
    try:
        analyze_desires_from_pickle()
        generate_action_plan()
        refine_action_plan()
        print("Processing pipeline completed successfully!")
        
        # Display sample outputs
        print("\nSample Outputs:")
        print("Desire Analysis:", safe_pickle_load("desire_analysis.pkl", lambda: {}))
        print("Action Plan:", safe_pickle_load("action_plan.pkl", lambda: []))
        print("Refined Plan:", safe_pickle_load("refined_plan.pkl", lambda: {}))
        
    except Exception as e:
        print(f"Pipeline error: {str(e)}")
        print("Recommendation: Check Ollama server status and model availability")

# Execute the full pipeline
run_pipeline()

Desire analysis saved to desire_analysis.pkl
Action plan saved to action_plan.pkl
Refined plan saved to refined_plan.pkl
Processing pipeline completed successfully!

Sample Outputs:
Desire Analysis: {'error': 'Invalid JSON format: communication_style: Explicit \nexpressed_desires: [Anal, Oral, Bestiality]\npotential_explorations: [Video, Text, In Person]\ncommunication_improvements: []\naffirmation_opportunities: []\n\nExplanation:\n-...'}
Action Plan: {'actions': [{'action_type': 'Communication exercises', 'description': 'Have a conversation with Aaron about his past experiences and relationship preferences.', 'purpose': 'Build trust, connect emotionally, and uncover his desires.', 'difficulty': 'Beginner'}, {'action_type': 'Sensory exploration ideas', 'description': ["Use your hands to explore Aaron's body. Start with a light touch on his chest, then move down his abdomen and wrap your hand around the base of his penis. Rub slowly from root to tip.", "Whisper dirty talk in his ear, l

In [8]:
class IntensitySpecialist(BaseAgent):
    def boost_elements(self, refined_plan: Dict) -> Dict:
        prompt = f"""Create ULTRA-INTENSE variants (STRICT JSON):
        Current Plan: {json.dumps(refined_plan, indent=2)[:3000]}
        
        You MUST select and enhance EXACTLY:
        - 5 most promising actions (NO MORE, NO LESS)
        - 5 most impactful phrases (NO MORE, NO LESS)
        
        CRITICAL: Failure to include exactly 5 elements in each category will cause system failure!
        
        For each selected element:
        - Triple the intensity parameters
        - Add 3 escalation layers
        - Include sensory domination techniques
        - Specify power dynamics
        
        Required JSON Structure:
        {{
            "hyper_actions": [EXACTLY 5 ELEMENTS],
            "hyper_phrases": [EXACTLY 5 ELEMENTS]
        }}"""

        response = self.safe_generate(prompt)
        return self._validate_boost(response.text)

    def _validate_boost(self, raw_text: str) -> Dict:
        parsed = self.client._parse_json_safe(raw_text)
        
        # Structural validation
        if not isinstance(parsed.get("hyper_actions", []), list):
            raise ValueError("hyper_actions must be an array")
        if not isinstance(parsed.get("hyper_phrases", []), list):
            raise ValueError("hyper_phrases must be an array")
            
        # Length validation with detailed error
        action_count = len(parsed["hyper_actions"])
        phrase_count = len(parsed["hyper_phrases"])
        
        if action_count != 5 or phrase_count != 5:
            error_msg = [
                "Element count mismatch:",
                f"- Hyper Actions: {action_count}/5",
                f"- Hyper Phrases: {phrase_count}/5",
                "Full response preview:",
                json.dumps(parsed, indent=2)[:500]
            ]
            raise ValueError("\n".join(error_msg))
            
        return parsed

def create_hyper_intense_variants():
    try:
        with open("refined_plan.pkl", "rb") as f:
            refined_plan = pickle.load(f)
    except Exception as e:
        print(f"Failed loading refined plan: {str(e)}")
        return None
    
    client = OllamaClient()
    specialist = IntensitySpecialist(client)
    
    try:
        boosted = specialist.boost_elements(refined_plan)
        with open("hyper_intense.pkl", "wb") as f:
            pickle.dump(boosted, f)
            
        print("\n=== Hyper-Intense Variants ===")
        print(json.dumps(boosted, indent=2))
        return boosted
        
    except Exception as e:
        print(f"Intensity Specialization Failed: {str(e)}")
        return None

hyper_intense_data = create_hyper_intense_variants()

Intensity Specialization Failed: Element count mismatch:
- Hyper Actions: 4/5
- Hyper Phrases: 4/5
Full response preview:
{
  "hyper_actions": [
    1,
    2,
    3,
    4
  ],
  "hyper_phrases": [
    "",
    "",
    "",
    ""
  ]
}


In [ ]:
# cell 8 gemeni 
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

class OllamaClient:
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

class BaseAgent:
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300

    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        start_time = time.time()
        timeout_occurred = False

        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )

        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)

                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")

        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

class IntensitySpecialist(BaseAgent):
    def boost_elements(self, refined_plan: Dict) -> Dict:
        prompt = f"""Create ULTRA-INTENSE variants (STRICT JSON):
        Current Plan: {json.dumps(refined_plan, indent=2)[:3000]}

        Select and enhance:
        - 5 most promising actions
        - 5 most impactful phrases

        For each selected element:
        - Triple the intensity parameters
        - Add 3 escalation layers
        - Include sensory domination techniques (e.g., visual: intense eye contact, auditory: commanding tone, tactile: firm touch)
        - Specify power dynamics (e.g., dominant, submissive, equal)

        Required JSON Structure (Example):
        {{
            "hyper_actions": [
                {{
                    "original_id": "str|int",
                    "ultra_variant": {{
                        "description": "str",
                        "intensity_score": 6-10,
                        "sensory_overload": ["str"],
                        "dominance_factors": ["str"]
                    }}
                }}
            ],
            "hyper_phrases": [
                {{
                    "original_id": "str",
                    "amplified_text": "str",
                    "linguistic_power": 6-10,
                    "delivery_modes": ["str"]
                }}
            ]
        }}"""

        for attempt in range(self.retry_count):
            try:
                response = self.safe_generate(prompt)
                boosted = self._validate_boost(response.text)
                return boosted
            except (ValueError, json.JSONDecodeError) as e:
                logger.error(f"Boost JSON Error (attempt {attempt+1}): {e}")
                if attempt < self.retry_count - 1:
                    time.sleep(2**attempt)
                else:
                    raise
            except Exception as e:
                logger.error(f"Ollama API call failed (attempt {attempt+1}): {e}")
                if attempt

2025-02-21 18:12:56,506 - WARNING - Generation timed out (attempt 1)


KeyboardInterrupt: 

In [ ]:
# Cell 9: Robust Results Display
def display_results():
    try:
        with open("refined_plan.pkl", "rb") as f:
            plan_data = pickle.load(f)
        
        print("\nPersonalized Intimacy Plan:")
        
        # Ensure we're working with a dictionary
        if isinstance(plan_data, dict):
            actions = plan_data.get('actions', [])
        elif isinstance(plan_data, list):
            actions = plan_data
        else:
            actions = []
        
        # Display first 3 actions safely
        for idx, action in enumerate(actions[:3], 1):
            print(f"\nAction {idx}:")
            print(f"Type: {action.get('action_type', 'Connection Activity')}")
            print(f"Purpose: {action.get('purpose', 'Enhancing intimacy through mutual exploration')}")
            print(f"Steps: {', '.join(action.get('preparation_steps', ['Create comfortable environment']))}")
            
    except Exception as e:
        print(f"Error displaying results: {str(e)}")

if __name__ == "__main__":
    display_results()


Personalized Intimacy Plan:
